In [1]:
import torch

In [2]:
#USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
BATCH_SIZE = 2
#MAX_STEPS = 300
FP16 = USE_CUDA
DEVICE = torch.device("cuda") if USE_CUDA else torch.device("cpu")
#DEVICE = torch.device("cpu")
print(f'{USE_CUDA   = }')
print(f'{BATCH_SIZE = }')

USE_CUDA   = False
BATCH_SIZE = 2


### Tokenizer

Now let's load our tokenizer from Hugging Face Hub

In [3]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(
    "phunc20/esperoberta-cased",
)
tokenizer.model_max_length

2023-06-26 21:05:24.180211: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 21:05:24.898707: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/phunc20/.config/miniconda3/envs/py3.10/lib
2023-06-26 21:05:24.898813: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/phunc20/.config/miniconda3/envs/py3.10/lib
2023-06-26 21:05:24.898824: W te

512

**(?)** Is it normal to have such a big `tokenizer.model_max_length`?  


In [4]:
tokenizer.vocab_size

30000

## Load The Dataset

In [5]:
from pathlib import Path

# dataset_dir = Path("/kaggle/input/vina-cased-chunked-dataset")
# dataset_dir.exists()

In [6]:
from datasets import load_dataset

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [7]:
dataset_repo = "phunc20/oscar_esperoberta-cased_dataset"
dataset = load_dataset(dataset_repo)
dataset

Found cached dataset parquet (/home/phunc20/.cache/huggingface/datasets/phunc20___parquet/phunc20--oscar_esperoberta-cased_dataset-19abbf324e90b9e8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 295376
    })
})

## The Model

In [8]:
from transformers import RobertaConfig

In [9]:
tokenizer.vocab_size

30000

In [10]:
tokenizer.bos_token_id, tokenizer.eos_token_id

(0, 2)

In [11]:
config = RobertaConfig(
    vocab_size=tokenizer.vocab_size,
    #max_position_embeddings=514,
    #num_attention_heads=12,
    #num_hidden_layers=6,
    #type_vocab_size=1,
)

**(?)** `type_vocab_size`? What is it?  

In [12]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print(f'{model.num_parameters():,d}')  # ~84 million parameters

109,112,880


Let's test its ability to guess masked sentences **before training**.

In [13]:
from transformers import pipeline

fill_mask = pipeline(
      "fill-mask",
      model=model,
      tokenizer=tokenizer,
)
# En 1831, havante 22 jarojn
result = fill_mask("""\
En 1831, havante 22 <mask>, ŝatanto de skaraboj sen direkto en vivo,
Darvino vojaĝis ĉirkaŭ la Tero en la ŝipo HMS Beagle,
dum kvin jaroj.\
""")
result

[{'score': 0.0002648446534294635,
  'token': 7505,
  'token_str': 'Par',
  'sequence': 'En 1831, havante 22Par, ŝatanto de skaraboj sen direkto en vivo, Darvino vojaĝis ĉirkaŭ la Tero en la ŝipo HMS Beagle, dum kvin jaroj.'},
 {'score': 0.0002455027133692056,
  'token': 28150,
  'token_str': ' uzur',
  'sequence': 'En 1831, havante 22 uzur, ŝatanto de skaraboj sen direkto en vivo, Darvino vojaĝis ĉirkaŭ la Tero en la ŝipo HMS Beagle, dum kvin jaroj.'},
 {'score': 0.00023882141977082938,
  'token': 10969,
  'token_str': ' gasto',
  'sequence': 'En 1831, havante 22 gasto, ŝatanto de skaraboj sen direkto en vivo, Darvino vojaĝis ĉirkaŭ la Tero en la ŝipo HMS Beagle, dum kvin jaroj.'},
 {'score': 0.00020915402274113148,
  'token': 27868,
  'token_str': ' nombri',
  'sequence': 'En 1831, havante 22 nombri, ŝatanto de skaraboj sen direkto en vivo, Darvino vojaĝis ĉirkaŭ la Tero en la ŝipo HMS Beagle, dum kvin jaroj.'},
 {'score': 0.00020576622046064585,
  'token': 4670,
  'token_str': 'ley',

## Data Collator

In [14]:
import collections

import numpy as np
from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [15]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15,
)

> Data collator expects as input **a list of dictionaries**

In [19]:
n_samples = 2
samples = [dataset["train"][i] for i in range(n_samples)]
# print(f'{samples = }')
for sample in samples:
    #import ipdb; ipdb.set_trace()
    del sample["word_ids"]

In [20]:
collated_samples = data_collator(samples)
collated_samples.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

Note that
- In `input_ids` there are about `mlm_probability` ratio of mask ids, `tokenizer.mask_token_id`
- In the corresponding position in `labels`, we can see the "correct" token id of the masked token. All the unmasked
  positions have `-100` as label, i.e. to be ignored

In [22]:
for chunk, label in zip(collated_samples["input_ids"],
                        collated_samples["labels"]):
    gt = []
    for c, l in zip(chunk, label):
        if c is None:
            continue
        if l == -100:
            gt.append(c)
        else:
            gt.append(l)
    gt = tokenizer.decode(
        gt,
        skip_special_tokens=True,
    )
    print(f'(label) {gt}')
    print()
    print(f"(chunk) '{tokenizer.decode(chunk)}'")

(label) Ĉu... preĝi | mediti | ricevi instigojn || kanti | muziki || informiĝi | legi | studi || prepari DiservonTemas pri kolekto de kristanaj kantoj, eldonita de Adolf Burkhardt inter 1974 kaj 1990 en dek kajeretoj. Ili estas reeldonitaj inter 1995 kaj 1998 de Bernhard Eichkorn en tri kajeroj, kies tria estas pliampleksigita per Dek Novaj Kantoj kaj suplemento, same de Adolf Burkhardt.En la dua kaj tria kajero oni adiciis 300 al la originaj kantonumeroj, por ke oni povu pli facile uzi la kajerojn kune kun la KELI-himnaro Adoru Kantante,

(chunk) '<s>Ĉu... preĝi | mediti | ricevi instigojn || kanti | muziki || informiĝi | legi | studi<mask> prepari Diservon</s><s>Temas pri kolekto de kristanaj kantoj, eldonita de Adolf Burkhardt inter 1974 kaj 1990 en dek kajeretoj. Ili estas<mask><mask> inter 1995 kaj 1998 de Bernhard Eichkorn<mask> tri kajeroj, kies tria estas<mask>igita<mask> Dek Novaj Kantoj<mask> suplemento, same sukcese<mask> Burkhardt.</s><s>En la dua kaj tria kajero oni adicii

**(?)** Why there are those `"�"`?

**Rmk.** Also, you might be interested in making sure only Latin-alphabet tokens are masked because masking characters from other languages, say, Chinese, wouldn't really help with our ultimate (Vietnamese) task.

In [23]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

## Downsampled Dataset

In [ ]:
dataset.remove_columns(["word_ids"])

In [24]:
train_size = 1_000
test_size = 100

downsampled_dataset = dataset["train"].train_test_split(
    train_size=train_size,
    test_size=test_size,
    #test_size=0.01,
    seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 100
    })
})

## `TrainingArguments`

In [25]:
from transformers import TrainingArguments

In [26]:
TrainingArguments?

Init signature:
TrainingArguments(
    output_dir: str,
    overwrite_output_dir: bool = False,
    do_train: bool = False,
    do_eval: bool = False,
    do_predict: bool = False,
    evaluation_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no',
    prediction_loss_only: bool = False,
    per_device_train_batch_size: int = 8,
    per_device_eval_batch_size: int = 8,
    per_gpu_train_batch_size: Optional[int] = None,
    per_gpu_eval_batch_size: Optional[int] = None,
    gradient_accumulation_steps: int = 1,
    eval_accumulation_steps: Optional[int] = None,
    eval_delay: Optional[float] = 0,
    learning_rate: float = 5e-05,
    weight_decay: float = 0.0,
    adam_beta1: float = 0.9,
    adam_beta2: float = 0.999,
    adam_epsilon: float = 1e-08,
    max_grad_norm: float = 1.0,
    num_train_epochs: float = 3.0,
    max_steps: int = -1,
    lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear',
    warmup_ratio: float = 0.0,
    w

In [30]:
from pathlib import Path
#data_dir = Path("/kaggle/workin")
data_dir = Path("/tmp")

In [34]:
model_name = "esperoberta-cased"
#(data_dir/model_name).mkdir(exist_ok=True)
# num_logging_times = 10

training_args = TrainingArguments(
    output_dir=data_dir/model_name,
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    do_train=True,
    do_eval=True,
    no_cuda=not USE_CUDA,
    #learning_rate=2e-5,
    learning_rate=1e-4,
    #num_train_epochs=1.0,
    num_train_epochs=5.0,
    #max_steps=MAX_STEPS,
    #max_steps=21_000,
    weight_decay=0.01,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    #push_to_hub=True,
    #fp16=True,  # only on CUDA dMAX_STEPS
    #logging_steps=MAX_STEPS//num_logging_times,
    #save_steps=MAX_STEPS//num_logging_times,
    fp16=FP16,
    logging_steps=200,
    save_steps=200,
    eval_steps=200,
    save_total_limit=4,
    #load_best_model_at_end=True,
    #remove_unused_columns=False,
    remove_unused_columns=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## Trainer

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Perplexity before training

In [36]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The following columns in the evaluation set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 2


>>> Perplexity: 36665.80


**(?)** `16`? Number of steps, right?  
**(R)** Yes, approximately: $\frac{1000}{64} \approx 16$

In [37]:
len(downsampled_dataset["test"]) // BATCH_SIZE

50

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
/home/phunc20/.config/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 109112880


Step,Training Loss,Validation Loss


In [43]:
ansi_code = {
    #"red":    "\033[91m",
    "red":    "\x1b[91m",
    "green":  "\033[92m",
    "yellow": "\033[93m",
    "blue":   "\033[94m",
    "pink":   "\033[95m",
    "teal":   "\033[96m",
    "grey":   "\033[97m",
    #"reset":    "\x1b[0m",
    "reset":    "\033[0m",
}

def infer_viz2(sample, k=5):
    inputs = data_collator([sample])
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    masked_text = tokenizer.decode(inputs["input_ids"][0])
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    #import ipdb; ipdb.set_trace()
    clone = torch.clone(inputs["input_ids"])
    clone[0, mask_token_index] = inputs["labels"][0, mask_token_index]
    labels = tokenizer.decode(clone[0])


    token_logits = model(**inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    # Pick the <mask> candidates with the highest logits
    top_k_tokens = torch.topk(mask_token_logits, k, dim=1).indices[0].tolist()

    #print("(masked)")
    #print(f'"{masked_text}"', end="\n\n")
    #
    #print("(gt)")
    #print(f'"{labels}"', end="\n\n")
    
    #print("(preds)")
    for token in top_k_tokens:
        replacement = f'{ansi_code["red"]}{tokenizer.decode([token])}{ansi_code["reset"]}'
        print(f"'{masked_text.replace(tokenizer.mask_token, replacement)}'")
        print()

In [44]:
import random

random.seed(42)
i = random.randint(0, len(downsampled_dataset["test"])-1)
sample = downsampled_dataset["test"][i]
print(sample)

{'input_ids': [11, 83, 11, 78, 324, 10227, 74, 11, 83, 4195, 11, 220, 213, 416, 11, 2314, 11, 1091, 11, 83, 11, 78, 11, 82, 16, 408, 2307, 379, 11410, 75, 11, 220, 389, 11, 82, 313, 2119, 17958, 11, 83, 30, 16512, 1739, 11, 220, 5728, 11, 69, 11, 82, 5805, 11, 83, 11, 82, 18, 2, 0, 4158, 11, 82, 1746, 11, 1579, 11, 73, 16, 213, 2865, 11, 83, 324, 213, 6064, 11, 1091, 11, 233, 11, 83, 404, 11, 361, 11, 220, 213, 12900, 11, 83, 11, 82, 18, 337, 684, 11, 24305, 11, 83, 11, 78, 2759, 4699, 237, 1236, 25540, 324, 4135, 11, 83, 4195, 11, 220, 391, 213, 403, 11, 69, 11, 82, 18, 2, 0, 954, 3293, 626, 395, 313, 2119], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [45]:
infer_viz2(sample)

' lao'j kun la la lao rigard'is la ĉe laest'ant' la laj'n, sed baldaŭ ek'reg lais ili' la mal'trankvil'o: Ramzes hav'is la'a'n vizaĝ'o'n.</s><s>Tio' la dir'int la la, la la'o kun la la laant'ar' la la'las'is la salon'o'n. La ĉef'pastr'o'j Herhor la Mefres kun tim'o rigard lais la la ali'a'n.</s><s>— Kial do ni la la'

'.o'j kun...o rigard'is la ĉe.est'ant'..j'n, sed baldaŭ ek'reg.is ili'. mal'trankvil'o: Ramzes hav'is.'a'n vizaĝ'o'n.</s><s>Tio'. dir'int.., la.'o kun la..ant'ar'..'las'is. salon'o'n. La ĉef'pastr'o'j Herhor. Mefres kun tim'o rigard.is. la ali'a'n.</s><s>— Kial do ni..'

',o'j kun,,,o rigard'is la ĉe,est'ant',,j'n, sed baldaŭ ek'reg,is ili', mal'trankvil'o: Ramzes hav'is,'a'n vizaĝ'o'n.</s><s>Tio', dir'int,,, la,'o kun la,,ant'ar',,'las'is, salon'o'n. La ĉef'pastr'o'j Herhor, Mefres kun tim'o rigard,is, la ali'a'n.</s><s>— Kial do ni,,'

' deo'j kun de de deo rigard'is la ĉe deest'ant' de dej'n, sed baldaŭ ek'reg deis ili' de mal'trankvil'o: Ramzes hav'is de'a'n vizaĝ'o'n

In [46]:
i = random.randint(0, len(downsampled_dataset["train"])-1)
sample = downsampled_dataset["train"][i]
print(sample)

{'input_ids': [970, 1413, 16, 2183, 8679, 223, 1769, 5049, 255, 299, 7187, 2984, 2219, 914, 520, 2, 0, 394, 8679, 223, 1769, 5049, 255, 299, 11795, 30, 2408, 814, 1017, 223, 1769, 1204, 23604, 4111, 343, 213, 492, 1567, 330, 213, 5613, 13232, 1159, 443, 6017, 308, 17, 69, 4975, 28825, 256, 1943, 18, 337, 8679, 5183, 213, 20744, 8944, 377, 1680, 237, 213, 17943, 6575, 3169, 18, 788, 343, 14977, 213, 2873, 219, 223, 2765, 8331, 16, 11428, 1664, 2380, 538, 213, 12136, 223, 3142, 20844, 16088, 24278, 18, 337, 970, 2084, 2766, 23317, 757, 436, 213, 1771, 2741, 223, 308, 17, 69, 6861, 7645, 12920, 4445, 450, 18, 699, 28, 65, 3350, 213, 26450, 9970, 16, 14977, 213, 1715, 219, 223, 2592, 8786, 16, 5183, 213, 673], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [48]:
infer_viz2(sample)

' la uzas, la Traktato la la la la lamalambiguigo).</s><s> la Traktato de Versajlo la france: Traité de la la la ) estis la plej grava el la packontraktoj la la la laa Mondmiliton al fino. La la finis la militan staton inter Germanio kaj la Aliancitaj ŝtatoj. la estis subskribita la 28an de junio 1919, ekzakte kvin jarojn la la murdo de arkiduko Franz Ferdinand. la la Centraj Potencoj sur la germana flanko de 1- la Mondmilito subskribis apartajn traktatojn. [8] Kvankam la armistico la subskribita la 11an de novembro 1918, finis la fa'

'. uzas,. Traktato.....malambiguigo).</s><s>. Traktato de Versajlo. france: Traité de... ) estis. plej grava el la packontraktoj....a Mondmiliton al fino. La. finis la militan staton inter Germanio kaj la Aliancitaj ŝtatoj.. estis subskribita la 28an de junio 1919, ekzakte kvin jarojn. la murdo de arkiduko Franz Ferdinand... Centraj Potencoj sur la germana flanko de 1-. Mondmilito subskribis apartajn traktatojn. [8] Kvankam la armistico. subskribita la 1